# Analiza Wielowymiarowa - Testy parametryczne i nieparametryczne - zajecia 2

In [ ]:
import os
os.getcwd() # oczekiwany .../AWXXXX/materialy/zajecia02
# mozna uzyc os.chdir("path") do zmiany

In [ ]:
import yaml
spec =  yaml.safe_load(open('../../spec.yaml'))

In [ ]:
# STATA
import stata_setup
stata_setup.config(spec["stata_path"], spec["stata_type"])
from pystata import stata

In [ ]:
import pandas as pd
import scipy
import numpy as np
import scipy.stats as stats
import sklearn
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
def f_test(x, y, alt: str = "two_sided"):
    """
    Calculates the F-test.
    :param x: array_like The first group of data
    :param y: The second group of data
    :param alt: The alternative hypothesis, one of "two_sided" (default), "greater" or "less"
    :return: a tuple with the F statistic value and the p-value.
    """
    x = np.array(x)
    y = np.array(y)
    df1 = len(x) - 1
    df2 = len(y) - 1
    f = x.var(ddof = 1) / y.var(ddof = 1)
    return {"statistic": f,
            "pval-greater": 1.0 - stats.f.cdf(f, df1, df2),
            "pval-less": stats.f.cdf(f, df1, df2),
            "pval-not equal": 2.0*(1.0 - stats.f.cdf(f, df1, df2))}

### Uzycie staty w jupyterach

%stata dla wywolan jednoliniowych
%%stata dla wywolan wieloliniowych

oba wywolania posiadaja dodatkowe argumenty ktore omowimy podczas zajec
%%stata -d dane -eret myeret

Obiekt stata z cala gama metod takich jak stata.run

### Podstawowe zbiory danych w STATA

In [ ]:
%stata help dta_examples

### Wczytywanie danych danych

In [ ]:
%stata sysuse auto, clear

In [ ]:
?stata.pdataframe_from_data

In [ ]:
auto = stata.pdataframe_from_data(missingval = np.NaN, valuelabel = True)
# i spowrotem
# stata.pdataframe_to_data(auto)

### Opis zawartosci zbioru danych

In [ ]:
%stata describe

### Ksiazka kodowa zbioru

Polecenie codebook pozwala nam zbadac charakterystyki per zmienna.

In [ ]:
%%stata
/* help codebook */
/* codebook */
codebook make

In [ ]:
auto.dtypes

### Wyswietlenie danych

Polecenie list bez dodatkowych zmiennych powinno byc wywolywane na mniejszych zbiorach.  
W przypadku wiekszych zbiorow przydatne bedzie uzycie dodatkowych argumentow zawezajacych wyszukanie.
W tym przypadku zawezimy wyszukanie samochodow do tych najciezszych.

In [ ]:
%%stata
/* help list */
/* list, table */
list if weight > 4500

In [ ]:
auto.query("weight > 4500")

Szybki przeglad zmiennych

In [ ]:
%%stata
/* help inspect */
/* inspect */
inspect weight

In [ ]:
%%stata
/* Statystyki opisowe */
summarize

In [ ]:
auto.describe()

## Testy parametryczne i nieparametryczne dla srednich

In [ ]:
# %stata help ttest
# ?stats.ttest_ind

test t dla srednich przy zalozeniu rownej wariancji
Warto zwrocic uwage ze roznica wynosi -312.2587 jendnak p-value dla wszytskich hipotez jest dosyc niskie gdyz odchylenie standardowe jest relatywnie wysokie.

In [ ]:
data = auto.groupby(["foreign"]).agg({"price": list})
{a: stats.ttest_ind(data.loc["Domestic", "price"], data.loc["Foreign", "price"], alternative = a) for a in ['two-sided', 'less', 'greater']}

In [ ]:
%%stata
ttest price, by(foreign)

test t dla srednich bez zalozenia o rownej wariancji

In [ ]:
stats.ttest_ind(data.loc["Domestic", "price"], data.loc["Foreign", "price"], equal_var=False)

In [ ]:
%stata ttest price, by(foreign) unequal

Histogram zmiennej price

In [ ]:
# matplotlib has to be installed
auto.price.hist(bins = 10)

In [ ]:
%stata histogram price, bin(10)

Argument "by" moze zostac uzyty na przestrzeni wielu funkcji

In [ ]:
auto.hist(column = "price", bins = 10, by = "foreign")

In [ ]:
%stata histogram price, bin(10) by(foreign)

/* Czy rozklad zmiennej jest zblizony do rozkladu normalnego, jesli nie, to jaki ma rozklad? */

In [ ]:
auto["lprice"] = np.log(auto.price)

In [ ]:
%stata gen lprice = ln(price)

/* Czy po transformacji zmienna ma rozklad zblizony do normalnego? */

In [ ]:
stats.shapiro(auto.lprice), stats.shapiro(auto.price)

In [ ]:
%stata swilk lprice price

Przeprowadzmy test dla przeksztalconej zmiennej

In [ ]:
%%stata
ttest lprice, by(foreign)
ttest lprice, by(foreign) unequal

In [ ]:
data = auto.groupby(["foreign"]).agg({"lprice": list})
stats.ttest_ind(data.loc["Domestic", "lprice"], data.loc["Foreign", "lprice"]),\
stats.ttest_ind(data.loc["Domestic", "lprice"], data.loc["Foreign", "lprice"], equal_var=False)

Test dla proporcji - Dla jednej proby

Prosze zwrocic uwage ze na dole wskazane sa Hipotezy alternatywne a nie zerowe.

In [ ]:
# for level 1 - Foreign
%stata prtest foreign==.4
#  Stata liczy p-value z rozkładu asymptotycznego (normalnego)

In [ ]:
stats.binomtest(auto.foreign.value_counts()["Foreign"], auto.shape[0], .4)

In [ ]:
{a: proportions_ztest(auto.foreign.value_counts()["Foreign"].sum(), auto.shape[0], .4, alternative = a) for a in ["smaller", "two-sided", "larger"]}

Sprawdzamy, czy udzial samochodow o cenie pow. 10.000$ jest taki sam jak zagranicznych

In [ ]:
%%stata
/* Tworzymy zmienna 0-1 o wartosci 1 dla ceny pow. 10.000$ */
gen price_10 = (price>10000)
prtest price_10==foreign

In [ ]:
auto["price_10"] = auto.price > 10000
proportions_ztest([auto["price_10"].sum(), auto.foreign.value_counts()["Foreign"]], [auto.shape[0], auto.shape[0]])

Testy dla wariancji - Test F

In [ ]:
%stata sdtest price, by(foreign)

In [ ]:
data = auto.groupby(["foreign"]).agg({"price": list})
f_test(data.loc["Domestic", "price"], data.loc["Foreign", "price"])

Test Levene

In [ ]:
%%stata
/* W0 - wartosc statystyki Levene */
/* W50 - wartosc statystyki dla testu, w ktorym zamiast sredniej wykorzystano mediane */
/* W10 - wartosc statystyki dla testu, w ktorym zamiast sredniej wykorzystano ucieta srednia  */
robvar price, by(foreign)

In [ ]:
stats.f_oneway(data.loc["Domestic", "price"], data.loc["Foreign", "price"])

Testy dla rozkladow - Test Kurskalla - Wallisa

In [ ]:
%stata kwallis price, by(foreign)

In [ ]:
stats.kruskal(data.loc["Domestic", "price"], data.loc["Foreign", "price"])

Test Kolmogorowa - Smirnowa

In [ ]:
%stata ksmirnov price, by(foreign)

In [ ]:
# - `two-sided`: The null hypothesis is that the two distributions are
#   identical, F(x)=G(x) for all x; the alternative is that they are not
#   identical.
# - `less`: The null hypothesis is that F(x) >= G(x) for all x; the
#   alternative is that F(x) < G(x) for at least one x.
# - `greater`: The null hypothesis is that F(x) <= G(x) for all x; the
#   alternative is that F(x) > G(x) for at least one x.
{ a:stats.ks_2samp(data.loc["Domestic", "price"], data.loc["Foreign", "price"], alternative = a) for a in ["less", "two-sided", "greater"]}
# roznice na pvalue

### Cwiczenie
Na postawie danych z 4 kwartalu 2009 roku bedziemy porownywac:  
1. place kobiet i mezczyzn w Polsce
2. wyksztalcenie kobiet i mezczyzn w Polsce
Prosze pobrac dane ze strony www.ekonometria.wne.uw.edu.pl i zaladowac do Staty  

Korzystajac z odpowiednich testow, prosze sprawdzic:
- Czy srednia placa kobiet jest rowna sredniej placy mezczyzn?
- Czy zroznicowanie plac kobiet jest rowne zroznicowaniu plac mezczyzn?
- Czy rozklad plac kobiet jest rowny rozkladowi plac mezczyzn?
- Kto jest lepiej wyksztalcony: kobiety czy mezczyzni?
- Czy zroznicowanie poziomu wyksztalcenia jest wieksze wsrod kobiet czy wsrod mezczyzn?
- Czy rozklad wyksztalcenia kobiet jest rowny rozkladowi wyksztalcenia mezczyzn?


In [ ]:
%stata clear

In [ ]:
# %stata use ...
# pd.read

Kilka ulatwien

In [ ]:
# %%stata
# /* Zmiana nazwy zmiennej */
# rename pl plec
# /* Utworzenie etykiety */
# label define plec_lab 1 "Mezczyzna" 2 "Kobieta"
# /* Nadanie etykiety wartosciom zmiennej */
# label values plec plec_lab
# /* utworzenie zmiennej placa, w bazie wartosc "99999" oznacza odmowe odpowiedzi */
# gen placa = P37 if (P37!=99999)

## Cwiczenie
Bedziemy pracowac na danych z badania Diagnoza Spoleczna z 2011 roku  
Prosze pobrac dane ze strony www.ekonometria.wne.uw.edu.pl i zaladowac zbior osob do Staty  
Prosze zapoznac sie z zawartoscia zbioru i odpowiedziec na pytania, przeprowadzajac odpowiednie testy
- Czy kobiety przecietnie wiecej waza od mezczyzn?
- Czy kobiety sa przecietnie wyzsze od mezczyzn? 
- Czy kobiety sa lepiej wyksztalcone od mezczyzn? 
- Czy wsrod osob o wyzszym wyksztalceniu kobiety przecietnie wiecej waza od mezczyzn? 
- Czy wsrod osob o wyzszym wyksztalceniu kobiety sa przecietnie wyzsze od mezczyzn? 
- Czy wsrod osob o wyzszym wyksztalceniu kobiety wiecej zarabiaja od mezczyzn? 
